In [39]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import json
import string
import re

from openpyxl import load_workbook

In [40]:
leadFeeder = pd.read_excel('./sheets/May Leadlander Data.xlsx')
wb = load_workbook(filename='./sheets/dtex-2022-06-03-10-08-29[4].xlsx')
adDaptive = pd.read_excel(
    './sheets/dtex-2022-06-03-10-08-29[4].xlsx', sheet_name=wb.sheetnames[1])

wb_target = load_workbook(filename='sheets/Target Account List (3).xlsx')
target_lvl1 = pd.read_excel(
    './sheets/Target Account List (3).xlsx', sheet_name=wb_target.sheetnames[0])
target_lvl2 = pd.read_excel(
    './sheets/Target Account List (3).xlsx', sheet_name=wb_target.sheetnames[1])
target_lvl3 = pd.read_excel(
    './sheets/Target Account List (3).xlsx', sheet_name=wb_target.sheetnames[2])

target_lvl1_companies = target_lvl1["Level 1"].to_numpy()
target_lvl2_companies = target_lvl2["Level 2"].to_numpy()
target_lvl3_companies = target_lvl3["Level 3"].to_numpy()
all_companies = np.concatenate([target_lvl1_companies, target_lvl2_companies, target_lvl3_companies])

adDaptive.columns = adDaptive.iloc[6]
adDaptive = adDaptive.iloc[7:, :]
adDaptive.reset_index(drop=True, inplace=True)

ad_companies = adDaptive[["Business Name", "Impressions"]].to_numpy()
lf_companies = leadFeeder[["Company", "Visit_PageViews"]].to_numpy()

In [41]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def clean_name(name):
    name = name.lower()
    name = name.translate(
        str.maketrans('', '', string.punctuation.replace("&", '')))
    name = name.strip()
    name = name.replace("  ", ' ')
    name = name.replace(' inc', '')
    name = name.replace(" llc", '')
    name = name.replace(" the", '')
    name = re.sub("\\[a-z]*", '', name)
    return name

In [42]:
def compute_cross_helper(comp_list, target, cross, key, list_type):
    remain = []
    for comp_arr in comp_list:
        comp = comp_arr[0]
        comp_name = clean_name(comp)
        sim = similar(target, comp_name)
        cond1 = similar(comp.lower().split()[0], target.split()[0]) > 0.9
        cond2 = comp.lower().startswith(target.split()[0]) or cond1
        if (sim > 0.85 and cond2):
            cross[key][list_type] += [{comp:comp_arr[1]}]
        else:
            remain.append(comp_arr)
    return cross, remain


def cross_helper_detail(comp_list, target, cross, key, list_type):
    remain = []
    for comp_arr in comp_list:
        comp = comp_arr[0]
        comp_name = clean_name(comp)
        sim = similar(target, comp_name)
        cond1 = target + " " in comp_name or comp_name in target
        cond2 = similar(comp.lower().split()[0], target.split()[0]) > 0.9
        cond3 = comp.lower().startswith(target.split()[0]) or cond2
        if sim > 0.4 and cond1 and cond3 and comp_name.startswith(target):
            cross[key][list_type] += [{comp:comp_arr[1]}]
        else:
            remain.append(comp_arr)
    return cross, remain

In [43]:
def compute_cross(target, adDaptive, leadFeeder, name):
    cross_list = {}
    [cross_list.setdefault(x, {"adDaptive": [], "LeadFeeder": []})
     for x in target]

    for comp in target:
        comp_name = clean_name(comp)
        cross_list, adDaptive = compute_cross_helper(
            adDaptive, comp_name, cross_list, comp, "adDaptive")
        cross_list, leadFeeder = compute_cross_helper(
            leadFeeder, comp_name, cross_list, comp, "LeadFeeder")

    for comp in target:
        comp_name = clean_name(comp)
        cross_list, adDaptive = cross_helper_detail(
            adDaptive, comp_name, cross_list, comp, "adDaptive")
        cross_list, leadFeeder = cross_helper_detail(
            leadFeeder, comp_name, cross_list, comp, "LeadFeeder")

    result_list = {}
    [result_list.setdefault(x, {"impressions": [], "page_views": []})
     for x in cross_list.keys()]
    
    for key, value in cross_list.items():
        result_list[key]["impressions"] = sum(x for vx in value["adDaptive"] for _, x in vx.items())
        result_list[key]["page_views"] = sum(x for vx in value["LeadFeeder"] for _, x in vx.items())

    # cross_list["remain"] = {
    #     "adDaptive": adDaptive, "LeadFeeder": leadFeeder}
    with open('./' + name + '.json', 'w') as f:
        json.dump(cross_list, f, indent=4)

    with open('./' + name + "_results" + '.json', 'w') as f:
        json.dump(result_list, f, indent=4)

In [44]:
# compute_cross(target_lvl1_companies, ad_companies, lf_companies, "lvl1")

In [45]:
# compute_cross(target_lvl2_companies, ad_companies, lf_companies, "lvl2")

In [46]:
# compute_cross(target_lvl3_companies, ad_companies, lf_companies, "lvl3")

In [47]:
compute_cross(all_companies, ad_companies, lf_companies, "all")

In [48]:
a = "CNO Financial Group"
b = "Actuarial Students'\u00e2\u20ac\u2039 Society at Macquarie University"

print(similar(clean_name(a), clean_name(b)))
print(clean_name(b))

0.2222222222222222
actuarial studentsâ€‹ society at macquarie university


In [49]:
print(b.lower().startswith(a.split()[0]) or similar(b.lower().split()[0], a.lower().split()[0]) > 0.85)
print(similar(b.lower().split()[0], a.lower().split()[0]))

False
0.16666666666666666
